- **CNN**은 2차원이나 그 이상 차원의 데이터 처리에 적합
- **DNN**은 이미지를 1차원 벡터로 변환하여 전 게층이 1차원 방식으로 신호를 처리하기 때문에 2차원 특성을 처리하는데에는 한계 존재
- **CNN**은 2차원 합성곱으로 각 노드를 처리하기 때문에 이미지에 더 적합

---

- 즉, **CNN**은 이미지의 높이와 넓이를 생각하면서 2차원 처리를 수행
- **CNN**은 합성곱 계층이 끝나면 맥스 풀링 계층을 이용

## 필기체 분류하는 CNN

In [1]:
import keras
from keras import models, layers
from keras import backend

'''
models: 모델링 방식에 해당되는 프레임 워크 : 연쇄방식 모델링 객체인 Sequential 사용
layers들의 패키지: Dense, Dropout, Flatten, Conv2D, MaxPooling2D

backend 서브 패키지: 딥러닝 엔진을 직접 제어
엔진에서 사용하는 시스템 파라미터 값들을 참조하거나 변경 가능
'''

본래 keras는 low-level은 핸들링 하지 못한다고 하지만, ```keras backend```를 이용하면 Tensorflow처럼 keras에서도 variable을 만들거나 연산 가능  


In [13]:
'''
super().__init__() = models.Sequential.__init__()와 동일
'''
class CNN(models.Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()
        
        self.add(layers.Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = input_shape))
        self.add(layers.MaxPooling2D(pool_size = (2,2)))
        self.add(layers.Dropout(0.25))
        self.add(layers.Flatten())
        self.add(layers.Dense(128, activation = 'relu'))
        self.add(layers.Dropout(0.5))
        self.add(layers.Dense(num_classes, activation = 'softmax'))
        
        self.compile(loss = keras.losses.categorical_crossentropy, optimizer = 'rmsprop', metrics = ['accuracy'])

In [14]:
from keras import datasets

class DATA():
    def __init__(self):
        num_classes = 10
        
        (x_train, y_train),(x_test, y_test) = datasets.mnist.load_data()
        
        img_rows, img_cols = x_train.shape[1:]
        
        # 흑백 이미지에는 채널 정보가 존재하지 않아서 입력 데이터의 차원을 하나 더 추가
        # image_data_format에 채널 위치 지정
        if backend.image_data_format() == 'channels_first':
            x_train = x_train.reshape(x_train.shape[0], 1 , img_rows, img_cols)
            x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
            input_shape = (1, img_rows, img_cols)
        else:
            x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
            x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols,1)
            input_shape = (img_rows, img_cols, 1)
        
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255
        
        y_train = keras.utils.to_categorical(y_train, num_classes)
        y_test = keras.utils.to_categorical(y_test, num_classes)
        
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test

In [15]:
# 학습 효과 분석
import matplotlib.pyplot as plt

In [16]:
def plot_acc(history, title=None):
    # summarize history for accuracy
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    if title is not None:
        plt.title(title)
    plt.ylabel('Accracy')
    plt.xlabel('Epoch')
    plt.legend(['Training data', 'Validation data'], loc=0)
    # plt.show()

In [17]:
def plot_loss(history, title=None):
    # summarize history for loss
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    if title is not None:
        plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training data', 'Validation data'], loc=0)
    # plt.show()

In [18]:
epochs = 10
batch_size = 128

In [19]:
data = DATA()
model = CNN(data.input_shape, data.num_classes)

In [25]:
history = model.fit(data.x_train, data.y_train, batch_size = batch_size, epochs = epochs, validation_split = 0.2)

Epoch 1/10
375/375 [==============================] - 16s 43ms/step - loss: 0.0568 - accuracy: 0.9834 - val_loss: 0.0603 - val_accuracy: 0.9847
Epoch 2/10
375/375 [==============================] - 17s 45ms/step - loss: 0.0596 - accuracy: 0.9833 - val_loss: 0.0601 - val_accuracy: 0.9852
Epoch 3/10
375/375 [==============================] - 17s 44ms/step - loss: 0.0617 - accuracy: 0.9827 - val_loss: 0.0530 - val_accuracy: 0.9852
Epoch 4/10
375/375 [==============================] - 16s 43ms/step - loss: 0.0594 - accuracy: 0.9825 - val_loss: 0.0566 - val_accuracy: 0.9864
Epoch 5/10
375/375 [==============================] - 16s 44ms/step - loss: 0.0616 - accuracy: 0.9830 - val_loss: 0.0527 - val_accuracy: 0.9868
Epoch 6/10
375/375 [==============================] - 16s 43ms/step - loss: 0.0607 - accuracy: 0.9836 - val_loss: 0.0620 - val_accuracy: 0.9854
Epoch 7/10
375/375 [==============================] - 17s 46ms/step - loss: 0.0633 - accuracy: 0.9833 - val_loss: 0.0554 - val_accuracy:

In [26]:
# 준비된 데이터를 data 인스턴스에 넣어두고 모델을 model인스턴스에 저장
score = model.evaluate(data.x_test, data.y_test)
print("Test Loss:", score[0])
print("Test accuracy:", score[1])

313/313 [==============================] - 1s 4ms/step - loss: 0.0522 - accuracy: 0.9859
Test Loss: 0.05224459245800972
Test accuracy: 0.9858999848365784


In [27]:
plot_acc(history)
plt.show()
plot_loss(history)
plt.show()

KeyError: 'acc'

## 컬러 이미지 분류하는 CNN

In [ ]:
from keras import datasets
import keras
assert keras.backend.image_data_format() == "channels_last"

In [ ]:
class Machine():
    def __init__(self, X, y, nb_classes=2, fig=True):
        self.nb_classes = nb_classes
        self.set_data(X, y)
        self.set_model()
        self.fig = fig

    def set_data(self, X, y):
        nb_classes = self.nb_classes
        self.data = DataSet(X, y, nb_classes)
        print('data.input_shape', self.data.input_shape)

    def set_model(self):
        nb_classes = self.nb_classes
        data = self.data
        self.model = CNN(nb_classes=nb_classes, in_shape=data.input_shape)
        # cnn_lenet(nb_classes=nb_classes, in_shape=data.input_shape)

    def fit(self, epochs=10, batch_size=128, verbose=1):
        data = self.data
        model = self.model

        history = model.fit(data.X_train, data.Y_train, batch_size=batch_size, epochs=epochs,
                            verbose=verbose, validation_data=(data.X_test, data.Y_test))
        return history

    def run(self, epochs=100, batch_size=128, verbose=1):
        data = self.data
        model = self.model
        fig = self.fig

        history = self.fit(epochs=epochs,
                           batch_size=batch_size, verbose=verbose)

        score = model.evaluate(data.X_test, data.Y_test, verbose=0)

        print('Confusion matrix')
        Y_test_pred = model.predict(data.X_test, verbose=0)
        y_test_pred = np.argmax(Y_test_pred, axis=1)
        print(metrics.confusion_matrix(data.y_test, y_test_pred))

        print('Test score:', score[0])
        print('Test accuracy:', score[1])

        # Save results
        suffix = sfile.unique_filename('datatime')
        foldname = 'output_' + suffix
        os.makedirs(foldname)
        skeras.save_history_history(
            'history_history.npy', history.history, fold=foldname)
        model.save_weights(os.path.join(foldname, 'dl_model.h5'))
        print('Output results are saved in', foldname)
          
        if fig:
            skeras.plot_acc_loss(history)

        self.history = history

        return foldname
    
def main():
    m = Machine()
    m.run()
    
if __name__ == "__main":
    main()
    

### LeNet
Conv2D 2개 + MaxPooling + Dense

In [30]:
from sklearn import model_selection, metrics
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import os
from keras import backend as K
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Input,Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [33]:
# CNN은 모델의 일종이므로, 케라스의 Model클래스를 상속해서 만듦
class CNN(Model): 
    def __init__(model, nb_classes, in_shape=None):
        model.nb_classes = nb_classes
        model.in_shape = in_shape
        model.build_model()
        super().__init__(model.x, model.y)
        model.compile()

    def build_model(model):
        nb_classes = model.nb_classes
        in_shape = model.in_shape

        x = Input(in_shape)

        h = Conv2D(32, kernel_size=(3, 3), activation='relu',
                   input_shape=in_shape)(x)
        h = Conv2D(64, (3, 3), activation='relu')(h)
        h = MaxPooling2D(pool_size=(2, 2))(h)
        h = Dropout(0.25)(h)
        h = Flatten()(h)
        z_cl = h

        h = Dense(128, activation='relu')(h)
        h = Dropout(0.5)(h)
        z_fl = h

        y = Dense(nb_classes, activation='softmax', name='preds')(h)

        model.cl_part = Model(x, z_cl)
        model.fl_part = Model(x, z_fl)

        model.x, model.y = x, y

    def compile(model):
        Model.compile(model, loss='categorical_crossentropy',
                      optimizer='adadelta', metrics=['accuracy'])

In [34]:
class DataSet:
    def __init__(self, X, y, nb_classes, scaling=True, test_size=0.2, random_state=0):
        """
        X is originally vector. Hence, it will be transformed
        to 2D images with a channel (i.e, 3D).
        """
        self.X = X
        self.add_channels()

        X = self.X
        # the data, shuffled and split between train and test sets
        X_train, X_test, y_train, y_test = model_selection.train_test_split(
            X, y, test_size=0.2, random_state=random_state)

        print(X_train.shape, y_train.shape)

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')

        if scaling:
            # scaling to have (0, 1) for each feature (each pixel)
            scaler = MinMaxScaler()
            n = X_train.shape[0]
            X_train = scaler.fit_transform(
                X_train.reshape(n, -1)).reshape(X_train.shape)
            n = X_test.shape[0]
            X_test = scaler.transform(
                X_test.reshape(n, -1)).reshape(X_test.shape)
            self.scaler = scaler

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_test.shape[0], 'test samples')

        # convert class vectors to binary class matrices
        Y_train = np_utils.to_categorical(y_train, nb_classes)
        Y_test = np_utils.to_categorical(y_test, nb_classes)

        self.X_train, self.X_test = X_train, X_test
        self.Y_train, self.Y_test = Y_train, Y_test
        self.y_train, self.y_test = y_train, y_test
        # self.input_shape = input_shape

    def add_channels(self):
        X = self.X

        if len(X.shape) == 3:
            N, img_rows, img_cols = X.shape

            if K.image_dim_ordering() == 'th':
                X = X.reshape(X.shape[0], 1, img_rows, img_cols)
                input_shape = (1, img_rows, img_cols)
            else:
                X = X.reshape(X.shape[0], img_rows, img_cols, 1)
                input_shape = (img_rows, img_cols, 1)
        else:
            input_shape = X.shape[1:]  # channel is already included.

        self.X = X
        self.input_shape = input_shape
